In [6]:
!gdown --id 0B7XkCwpI5KDYNlNUTTlSS21pQmM

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
From (redirected): https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM&confirm=t&uuid=a152840d-5dd3-4eff-89ce-2516b6759ee6
To: /content/GoogleNews-vectors-negative300.bin.gz
100% 1.65G/1.65G [00:12<00:00, 136MB/s]


In [1]:
!pip install ffmpeg-python

In [2]:
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [4]:
!pip install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 51.1 MB/s eta 0:00:00


In [8]:
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
import speech_recognition as sr

# Download the Word2Vec model
# !wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
EMBEDDING_FILE = '/content/GoogleNews-vectors-negative300.bin.gz'

# Preprocessing question paper
with open("questionpaper.txt", "r") as file:
    data = file.read().replace("\n", " ")
data = data.split(" ")
question_paper = [" ".join(data[i:i+5]) for i in range(0, len(data), 5)]

# Define the similarity class
class DocSim:
    def __init__(self, w2v_model, stopwords=None):
        self.w2v_model = w2v_model
        self.stopwords = stopwords if stopwords is not None else []

    def vectorize(self, doc):
        words = [w.lower() for w in doc.split() if w not in self.stopwords]
        word_vecs = [self.w2v_model[word] for word in words if word in self.w2v_model]
        return np.mean(word_vecs, axis=0) if word_vecs else np.zeros(self.w2v_model.vector_size)

    def _cosine_sim(self, vecA, vecB):
        return np.dot(vecA, vecB) / (np.linalg.norm(vecA) * np.linalg.norm(vecB) + 1e-9)

    def calculate_similarity(self, source_doc, target_docs, threshold=0.6):
        source_vec = self.vectorize(source_doc)
        results = [{"score": self._cosine_sim(source_vec, self.vectorize(doc)), "doc": doc} for doc in target_docs]
        return sorted([res for res in results if res["score"] > threshold], key=lambda k: k["score"], reverse=True)

# Load the model and stopwords
model = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
with open("stopwords_en.txt", 'r') as fh:
    stopwords = fh.read().split(",")
ds = DocSim(model, stopwords=stopwords)

# Recognizing audio
count = 0
sound = True

while sound:
    audio, sr_rate = get_audio()  # Capture audio
    recognizer = sr.Recognizer()
    try:
        audio_text = recognizer.recognize_google(sr.AudioData(audio.tobytes(), sr_rate, 2))
        print(f"Recognized Text: {audio_text}")
        sim_scores = ds.calculate_similarity(audio_text, question_paper)

        if sim_scores and sim_scores[0]['score'] > 0.6:
            count += 1
        print(f"Similarity Scores: {sim_scores}")
        print(f"Number of Detected Cheating Attempts: {count}")
    except Exception as e:
        print(f"Error recognizing audio: {e}")


Recognized Text: hello how are you hello
Similarity Scores: []
Number of Detected Cheating Attempts: 0


Recognized Text: no cheating Atkins Hospital
Similarity Scores: []
Number of Detected Cheating Attempts: 0


Recognized Text: restate the difference between simulation and modeling
Similarity Scores: []
Number of Detected Cheating Attempts: 0


Recognized Text: call Jenna difference between simulation and body illustrate the difference between simulation and modeling outline the difference between analytical and numerical simulation explain continuous system with a suitable example list some major industries where simulation is used demonstrate how we can offset the disadvantage of simulation
Similarity Scores: [{'score': 0.678280143730433, 'doc': 'major industries where simulation is'}, {'score': 0.6659491447797637, 'doc': 'simulation and modelling.[CO1-Understanding] 2. Outline'}, {'score': 0.6120555236505144, 'doc': 'the difference between analytical and'}]
Number of Detected Cheating Attempts: 1


KeyboardInterrupt: 